In [1]:
import torch
torch.set_default_tensor_type(torch.FloatTensor)
from torch.autograd import Variable
from torch.nn import functional as F
from torchmetrics import Accuracy, Recall, Precision, Specificity, ConfusionMatrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_recall_curve, average_precision_score,roc_curve, auc, precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt
from collections import Counter
import random
import math
import torch.optim as optim
from tabulate import tabulate
from ray import tune

from fractions import Fraction

In [2]:
# Create Tensors to hold input and outputs.
# beta_1=np.append(np.array([4,4,-8,-4,6,8,-4,4,-6,-8]),np.zeros(90))
# beta_2=np.append(np.array([-2,2,4,-2,3,-4,2,-2,-3,4]),np.zeros(90))
# beta_3=np.append(np.array([-3,-3,6,-3,-4.5,-6,3,-3,4.5,6]),np.zeros(90))

#第二组参数

beta_1=np.append(np.array([4,4,-8,-4,6,-4,8,4,-6,-8]),np.zeros(90))
beta_2=np.append(np.array([-2,2,4,-2,3,2,-4,-2,-3,4]),np.zeros(90))
beta_3=np.append(np.array([-3,-3,6,-3,-4.5,3,-6,-3,4.5,6]),np.zeros(90))

beta_1=beta_1.reshape(-1,1)
beta_2=beta_2.reshape(-1,1)
beta_3=beta_3.reshape(-1,1)

n1=200
n2=200
n3=200
p=100
corval=0
prior=[11,12,13,14,15]

def sigmoid(X):
    return .5 * (1 + np.tanh(.5 * X))

def generate_data(corval,beta,n,p):
    mean=np.zeros(p)
    sigma=np.array([[corval**abs(i-j) for i in range(p)] for j in range(p)])
    x=torch.tensor(np.random.multivariate_normal(mean=mean,cov=sigma,size=n))
    Pi_test= sigmoid(x@beta)
    y=np.random.binomial(1,Pi_test.ravel(),n)
    x=x.to(torch.float32)
    #y=torch.tensor(y,dtype=torch.double)
    y=torch.tensor(y,dtype=torch.float)
    #y=y.to(torch.float32)
    return x,y
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

def cubic_root(x):
    return math.copysign(math.pow(abs(x), 1.0/3.0), x)

In [3]:
def train_model(config):
    split_num=5
    kf = KFold(n_splits=split_num)



    sum_test_BCE_loss=0
    for idx_train,idx_test in kf.split(inputs):
        lambda1,lambda2,lr,ga= config["lambda1"], config["lambda2"],config["lr"],config["ga"]
        model=MLP(seed=1)
        max_iteration=1000
        learning_rate=lr
        params_to_optimize=list(model.parameters())
        optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
        loss_fn = torch.nn.BCELoss()
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=ga)

        BCE=[]

        for t in range(max_iteration):
            optimizer.zero_grad() # renew optimizer
            out, layer1_out, layer2_out= model(inputs[idx_train])

            # extract parameters
            #[:-1] for leaving out bias term#
            model_all_linear1_params = model.linear1
            model_all_linear2_params= torch.cat([x.view(-1) for x in model.linear2.parameters()][:-1])
            model_all_linear3_params= torch.cat([x.view(-1) for x in model.linear3.parameters()][:-1])

    


            # compute loss
            all_linear1_params=model_all_linear1_params.reshape(1,-1)
            all_linear23_params=torch.cat((model_all_linear2_params,model_all_linear3_params),0)
            
            non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)
            BCE_loss = loss_fn(out, targets[idx_train].reshape(-1,1))


#             l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)
            l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

            l2_regularization = lambda2* torch.sum(all_linear23_params.pow(2))

            loss =BCE_loss+l1_regularization +l2_regularization

            # record loss
            BCE.append(BCE_loss.item())

            # compute derivative
            loss.backward()

            # gradient descent
            optimizer.step()

            # learning rate decay
            scheduler.step()
        
        
        test_out, test_layer1_out, test_layer2_out= model(train_x[idx_test])

        
        test_BCE_loss = loss_fn(test_out, targets[idx_test].reshape(-1,1))

        
        sum_test_BCE_loss+=test_BCE_loss.item()

    #print("loss: ",)
    tune.report(my_test_BCE_loss=sum_test_BCE_loss/split_num)

In [4]:
def train_model_2(config):
    split_num=5
    kf = KFold(n_splits=split_num)
    
    
    lambda1,lambda2,lr,eta = config["lambda1"], config["lambda2"],config["lr"],config["eta"]
    inputs=train_x
    targets=(1-eta)*train_y+eta*y_prior
    
    
    sum_test_BCE_loss=0
    for idx_train,idx_test in kf.split(inputs):
       
        
        model=MLP(seed=1)

        max_iteration=1000
        learning_rate=lr
        params_to_optimize=list(model.parameters())
        optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
        loss_fn = torch.nn.BCELoss()
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

        BCE=[]

        for t in range(max_iteration):
            optimizer.zero_grad() # renew optimizer
            out, layer1_out, layer2_out,layer3_out= model(inputs[idx_train])


            # extract parameters
            #[:-1] for leaving out bias term#
            model_all_linear1_params = model.linear1
            model_all_linear2_params= torch.cat([x.view(-1) for x in model.linear2.parameters()][:-1])
            model_all_linear3_params= torch.cat([x.view(-1) for x in model.linear3.parameters()][:-1])
            model_all_linear4_params= torch.cat([x.view(-1) for x in model.linear4.parameters()][:-1])
    


            # compute loss
            all_linear1_params=model_all_linear1_params.reshape(1,-1)
            all_linear234_params=torch.cat((model_all_linear2_params,model_all_linear3_params, model_all_linear4_params),0)

            BCE_loss = loss_fn(out, targets[idx_train].reshape(-1,1))
            
            non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)
#             l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)
            l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

            l2_regularization = lambda2* torch.sum(all_linear234_params.pow(2))
            loss =BCE_loss+l1_regularization +l2_regularization

            # record loss
            BCE.append(BCE_loss.item())

            # compute derivative
            loss.backward()

            # gradient descent
            optimizer.step()

            # learning rate decay
            scheduler.step()
        
        
        test_out, test_layer1_out, test_layer2_out,test_layer3_out= model(train_x[idx_test])

        test_BCE_loss = loss_fn(test_out, targets[idx_test].reshape(-1,1))
        
        sum_test_BCE_loss+=test_BCE_loss.item()


    #print("loss: ",)
    tune.report(my_test_BCE_loss=sum_test_BCE_loss/split_num)

In [5]:
TPR=[]
TNR=[]
GM=[]
accuracy=[]
precision=[]
f1=[]=[]
vsSEN=[]
vsSPE=[]
vsGM=[]
vsMR=[]
vsCCR=[]


vs = np.zeros((1,100))

In [6]:
for i in range(10):
    setup_seed(i)
       
    x1,y1=generate_data(corval,beta_1,n1,p)
    x2,y2=generate_data(corval,beta_2,n2,p)
    x3,y3=generate_data(corval,beta_3,n3,p)

    x= torch.cat((x1,x2), dim=0)
    x= torch.cat((x,x3), dim=0)

    y= torch.cat((y1,y2), dim=0)
    y= torch.cat((y,y3), dim=0)
    
    train_x, test_x, train_y, test_y= train_test_split(x,y, test_size=0.2, random_state=1)

    
    # switch training set
    inputs,targets = train_x,train_y

    # switching testing set
    test_inputs, test_targets = test_x,test_y
    
################## first step######################################  
    prior=[11,12,13,14,15]

# choose lambda1,lambda2 by ray tune
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1 = torch.nn.Parameter(torch.randn(p))
            self.linear2 = torch.nn.Linear(p,10)
            self.linear3 = torch.nn.Linear(10,1)

        def forward(self, x):
            layer1_out = self.linear1*x
            layer2_out = F.relu(self.linear2(layer1_out))
            out= torch.sigmoid(self.linear3(layer2_out))
            return out, layer1_out, layer2_out


    config={
            "lr":tune.choice([0.6,0.7,0.8]),
            "lambda1": tune.choice([0.03,0.04,0.05]),
            "lambda2": tune.choice([0.001,0.002,0.003]),
            "ga":tune.choice([0.3,0.4,0.5])
        }

    result = tune.run(
            train_model,  
            config=config,
            num_samples=30)
    
################## second step########################################    

    final_lambda=result.get_best_config('my_test_BCE_loss',mode='min')
    lambda1,lambda2,lr,ga=final_lambda["lambda1"],final_lambda["lambda2"],final_lambda["lr"],final_lambda["ga"]
    learning_rate=lr
    max_iteration=1000
    
    BCE=[]
    
    model=MLP(seed=1)

    loss_fn = torch.nn.BCELoss()
    params_to_optimize=list(model.parameters())
    optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
    
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=ga)

    loss_record=[]
    
    for t in range(max_iteration):
    
        # renew optimizer
        optimizer.zero_grad()

        # forward propagate
        out, layer1_out, layer2_out= model(inputs)



        # extract parameters
        #[:-1] for leaving out bias term#
        model_all_linear1_params = model.linear1
        model_all_linear2_params= torch.cat([x.view(-1) for x in model.linear2.parameters()][:-1])
        model_all_linear3_params= torch.cat([x.view(-1) for x in model.linear3.parameters()][:-1])

        # compute loss

        all_linear1_params=model_all_linear1_params.reshape(1,-1)
        all_linear23_params=torch.cat((model_all_linear2_params,model_all_linear3_params),0)

        BCE_loss = loss_fn(out, targets.reshape(-1,1))
        
        non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)
    #     l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)

        l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

        l2_regularization = lambda2 * torch.sum(all_linear23_params.pow(2))

        loss =BCE_loss+l1_regularization +l2_regularization

        loss_record.append(loss.item())
        # record loss
        BCE.append(BCE_loss.item())

        # compute derivative
        loss.backward()

        # gradient descent
        optimizer.step()

        # learning rate decay
        scheduler.step()
    
################## Y_prior##################    
    y_prior= (model(train_x)[0]).detach().squeeze()

    
    
################## third step##################

    #no prior
    prior=[]
    # choose lambda1,lambda2 by ray tune
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1 = torch.nn.Parameter(torch.randn(p))
            self.linear2 = torch.nn.Linear(p,10)
            self.linear3 = torch.nn.Linear(10,10)
            self.linear4 = torch.nn.Linear(10,1)
        def forward(self, x):
            layer1_out = self.linear1*x
            layer2_out = F.relu(self.linear2(layer1_out))
            layer3_out = F.relu(self.linear3(layer2_out))
            out= torch.sigmoid(self.linear4(layer3_out))
            return out, layer1_out, layer2_out,layer3_out

    config={
        "lambda1": tune.choice([0.03,0.04,0.05]),
        "lambda2": tune.choice([0.001,0.002,0.003]),
         "eta": tune.choice([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]),
         "lr":tune.choice([0.6,0.7,0.8])
                            }

    result = tune.run(
            train_model_2,  
            config=config,
            num_samples=160)
    

################## fourth step##################

    #no prior
    prior=[]
    #0.4,0.4
    # set hyperparamters
    final_parameter=result.get_best_config('my_test_BCE_loss',mode='min')
    lambda1,lambda2,eta,lr=final_parameter["lambda1"],final_parameter["lambda2"],final_parameter["eta"],final_parameter["lr"]
    learning_rate=lr
    max_iteration=1000

    inputs=train_x
    targets=(1-eta)*train_y+eta*y_prior

    # switching testing set
    test_inputs, test_targets = test_x,test_y

    # record loss descent
    BCE=[]


    # main nn object
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1 = torch.nn.Parameter(torch.randn(p))
            self.linear2 = torch.nn.Linear(p,10)
            self.linear3 = torch.nn.Linear(10,10)
            self.linear4 = torch.nn.Linear(10,1)

        def forward(self, x):
            layer1_out = self.linear1*x
            layer2_out = F.relu(self.linear2(layer1_out))
            layer3_out = F.relu(self.linear3(layer2_out))
            out= torch.sigmoid(self.linear4(layer3_out))
            return out, layer1_out, layer2_out,layer3_out



    model=MLP(seed=1)

    loss_fn = torch.nn.BCELoss()
    params_to_optimize=list(model.parameters())
    optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)

    # learning rate decay scheme
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

    loss_record=[]
    # loop for max_iteration times
    for t in range(max_iteration):

        # renew optimizer
        optimizer.zero_grad()

        # forward propagate
        out, layer1_out, layer2_out,layer3_out= model(inputs)



        model_all_linear1_params = model.linear1
        model_all_linear2_params= torch.cat([x.view(-1) for x in model.linear2.parameters()][:-1])
        model_all_linear3_params= torch.cat([x.view(-1) for x in model.linear3.parameters()][:-1])
        model_all_linear4_params= torch.cat([x.view(-1) for x in model.linear4.parameters()][:-1])

        # compute loss

        all_linear1_params=model_all_linear1_params.reshape(1,-1)
        all_linear234_params=torch.cat((model_all_linear2_params,model_all_linear3_params, model_all_linear4_params),0)

        BCE_loss = loss_fn(out, targets.reshape(-1,1))

        non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)

        l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))
        l2_regularization = lambda2 * torch.sum(all_linear234_params.pow(2))
        loss =BCE_loss+l1_regularization +l2_regularization

        loss_record.append(loss.item())
        # record loss
        BCE.append(BCE_loss.item())

        # compute derivative
        loss.backward()

        # gradient descent
        optimizer.step()

        # learning rate decay
        scheduler.step()
        
    best_trial = result.get_best_trial("my_test_BCE_loss", "min", "last")
    final_parameter=result.get_best_config('my_test_BCE_loss',mode='min')
    print(final_parameter)
    
####################################   prediction ####################################      

    prediction= (model(test_x)[0]>0.5).clone().int()
    target= test_y.reshape(-1,1).int()


    prediction=prediction.tolist()
    target=target.tolist()

    final_matrix=confusion_matrix(target,prediction)
    TPR.append(recall_score(target,prediction))#TPR
    TNR.append(final_matrix[0,0]/(sum(final_matrix[0,:])))#TNR
    GM.append(math.sqrt(TPR[-1]*TNR[-1]))
    accuracy.append(accuracy_score(target,prediction))
    precision.append(precision_score(target,prediction))
    f1.append(f1_score(target,prediction))

    
    
####################################   variable selection ####################################     
    model_weight=model_all_linear1_params.detach().numpy().copy().reshape(1,-1)

    threshold=0.12
    mcl_w=np.zeros(p)

    for j in range(p):
        mcl_w[j]=model_weight[0,j]

    max_mcl_w=max(abs(mcl_w))
    for j in range(p):
        if (abs(mcl_w[j]))<=threshold*max_mcl_w:
            model_weight[0,j]=0
        else:
            model_weight[0,j]=model_weight[0,j]
    
    index=np.nonzero(model_weight)[1]
    
    beta=np.array(beta_1)
    trueindex=np.where(beta!=0)[0]
    trueindex=trueindex.tolist()
    trueindex=set(trueindex)
    
    index=np.nonzero(model_weight)[1]
    index=index.tolist()
    index=set(index)
    
    TP=len(trueindex.intersection(index))
    FP=len(index.difference(trueindex))

    FN=len(trueindex)-TP
    TN=p-len(trueindex)-FP
    
    vsSEN.append(TP/(TP+FN))
    vsSPE.append(TN/(TN+FP))
    vsGM.append(math.sqrt(vsSEN[-1]*vsSPE[-1]))
    vsMR.append((FP+FN)/(TP+FN+TN+FP))
    vsCCR.append(1-vsMR[-1])
    
    print(i)

2022-11-17 19:58:30,829	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8270 
2022-11-17 19:58:31,723	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,ga,lambda1,lambda2,lr,iter,total time (s),my_test_BCE_loss
train_model_28091_00000,TERMINATED,10.224.0.12:27842,0.3,0.03,0.003,0.8,1,7.48812,0.743512
train_model_28091_00001,TERMINATED,10.224.0.12:27898,0.5,0.03,0.001,0.8,1,7.76388,0.824996
train_model_28091_00002,TERMINATED,10.224.0.12:27900,0.4,0.05,0.001,0.6,1,7.30738,0.718449
train_model_28091_00003,TERMINATED,10.224.0.12:27902,0.4,0.05,0.002,0.8,1,7.84204,0.716667
train_model_28091_00004,TERMINATED,10.224.0.12:27905,0.4,0.04,0.002,0.7,1,7.63083,0.734617
train_model_28091_00005,TERMINATED,10.224.0.12:27906,0.4,0.03,0.002,0.8,1,6.63751,0.774658
train_model_28091_00006,TERMINATED,10.224.0.12:27907,0.4,0.04,0.002,0.7,1,7.58826,0.734617
train_model_28091_00007,TERMINATED,10.224.0.12:27910,0.4,0.03,0.001,0.7,1,7.78061,0.775101
train_model_28091_00008,TERMINATED,10.224.0.12:27912,0.5,0.03,0.003,0.8,1,7.33303,0.751882
train_model_28091_00009,TERMINATED,10.224.0.12:27915,0.5,0.03,0.001,0.6,1,7.74642,0.784014


(raylet) [2022-11-17 19:58:40,717 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 25300992; capacity: 29394726912. Object creation will fail if spilling is required.


Result for train_model_28091_00000:
  date: 2022-11-17_19-58-41
  done: false
  experiment_id: 691994ee4c164437b9f165827b4f5255
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.7435117483139038
  node_ip: 10.224.0.12
  pid: 27842
  time_since_restore: 7.488123893737793
  time_this_iter_s: 7.488123893737793
  time_total_s: 7.488123893737793
  timestamp: 1668686321
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: '28091_00000'
  warmup_time: 0.004517793655395508
  
Result for train_model_28091_00000:
  date: 2022-11-17_19-58-41
  done: true
  experiment_id: 691994ee4c164437b9f165827b4f5255
  experiment_tag: 0_ga=0.3000,lambda1=0.0300,lambda2=0.0030,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.7435117483139038
  node_ip: 10.224.0.12
  pid: 27842
  time_since_restore: 7.488123893737793
  time_this_iter_s: 7.488123893737793
  time_total_s: 7.488123893737793
  timestamp: 1668686321
  timesteps

Result for train_model_28091_00028:
  date: 2022-11-17_19-58-44
  done: false
  experiment_id: b154950789fd493bb3da1daa82970e84
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.7323945760726929
  node_ip: 10.224.0.12
  pid: 27952
  time_since_restore: 7.292848825454712
  time_this_iter_s: 7.292848825454712
  time_total_s: 7.292848825454712
  timestamp: 1668686324
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: '28091_00028'
  warmup_time: 0.003782987594604492
  
Result for train_model_28091_00028:
  date: 2022-11-17_19-58-44
  done: true
  experiment_id: b154950789fd493bb3da1daa82970e84
  experiment_tag: 28_ga=0.5000,lambda1=0.0500,lambda2=0.0010,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.7323945760726929
  node_ip: 10.224.0.12
  pid: 27952
  time_since_restore: 7.292848825454712
  time_this_iter_s: 7.292848825454712
  time_total_s: 7.292848825454712
  timestamp: 1668686324
  timestep

...

2022-11-17 20:03:20,367	WARNING util.py:220 -- The `start_trial` operation took 2.006 s, which may be a performance bottleneck.
2022-11-17 20:03:20,381	WARNING trial_runner.py:879 -- Trial Runner checkpointing failed: [Errno 28] No space left on device
2022-11-17 20:03:20,399	ERROR ray_trial_executor.py:562 -- Trial train_model_2_6f1c2_00276: Unexpected error starting runner.
Traceback (most recent call last):
  File "/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/ray/tune/execution/ray_trial_executor.py", line 555, in start_trial
    return self._start_trial(trial)
  File "/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/ray/tune/execution/ray_trial_executor.py", line 458, in _start_trial
    runner = self._setup_remote_runner(trial)
  File "/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/ray/tune/execution/ray_trial_executor.py", line 323, in _setup_remote_runner
    trial.init_logdir()
  File "/home/rd/anaconda3/envs/envJK/lib/python3.10/site-package

2022-11-17 20:03:26,449	ERROR ray_trial_executor.py:537 -- Trial train_model_2_6f1c2_00278: Error stopping runner.
Traceback (most recent call last):
  File "/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/ray/tune/execution/ray_trial_executor.py", line 555, in start_trial
    return self._start_trial(trial)
  File "/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/ray/tune/execution/ray_trial_executor.py", line 458, in _start_trial
    runner = self._setup_remote_runner(trial)
  File "/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/ray/tune/execution/ray_trial_executor.py", line 323, in _setup_remote_runner
    trial.init_logdir()
  File "/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/ray/tune/experiment/trial.py", line 567, in init_logdir
    logdir_path.mkdir(parents=True, exist_ok=True)
  File "/home/rd/anaconda3/envs/envJK/lib/python3.10/pathlib.py", line 1173, in mkdir
    self._accessor.mkdir(self, mode)
OSError: [Errno 28] No space lef

Result for train_model_2_6f1c2_00207:
  date: 2022-11-17_20-01-38
  done: false
  experiment_id: dc26ed9cc5aa4df799507883f4b976a5
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.6935677766799927
  node_ip: 10.224.0.12
  pid: 4747
  time_since_restore: 8.224634408950806
  time_this_iter_s: 8.224634408950806
  time_total_s: 8.224634408950806
  timestamp: 1668686498
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 6f1c2_00207
  warmup_time: 0.0046749114990234375
  


TuneError: Traceback (most recent call last):
  File "/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/ray/tune/execution/trial_runner.py", line 839, in _wait_and_handle_event
    self._on_training_result(
  File "/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/ray/tune/execution/trial_runner.py", line 964, in _on_training_result
    self._process_trial_results(trial, result)
  File "/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/ray/tune/execution/trial_runner.py", line 1048, in _process_trial_results
    decision = self._process_trial_result(trial, result)
  File "/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/ray/tune/execution/trial_runner.py", line 1103, in _process_trial_result
    self._callbacks.on_trial_result(
  File "/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/ray/tune/callback.py", line 329, in on_trial_result
    callback.on_trial_result(**info)
  File "/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/ra

(raylet) [2022-11-17 20:03:31,039 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 0; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-17 20:03:41,047 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 0; capacity: 29394726912. Object creation will fail if spilling is required.


In [7]:
####################################   prediction ####################################   
from numpy import *
mean(TPR)
mean(TNR)
mean(GM)
mean(accuracy)
mean(precision)
mean(f1)


0.34903613335816724

0.7426415797317436

0.3404329085684977

0.5616666666666666

0.3457662337662338

0.34272723092134005

In [11]:
####################################   variable selection ####################################   
mean(vsSEN)
mean(vsSPE)
mean(vsGM)
mean(vsMR)
mean(vsCCR)

0.4

0.5844444444444445

0.2677871785461234

0.31

0.6900000000000001

In [9]:
vs

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [7]:
i

(raylet) [2022-11-17 20:03:51,054 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 7938048; capacity: 29394726912. Object creation will fail if spilling is required.


1

(raylet) [2022-11-17 20:04:01,061 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 13680640; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-17 20:04:11,069 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 23023616; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-17 20:04:21,077 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 29745152; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-17 20:04:31,085 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 29413376; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-17 20:09:11,307 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 24743936; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-17 20:09:21,315 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 24649728; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-17 20:09:31,322 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 24309760; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-17 20:09:41,330 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 23900160; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-17 20:14:21,613 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 19349504; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-17 20:14:31,621 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 19013632; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-17 20:14:41,629 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 18591744; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-17 20:14:51,637 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 18522112; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-17 20:19:31,870 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 13897728; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-17 20:19:41,877 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 13484032; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-17 20:19:51,885 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 13426688; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-17 20:20:01,892 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 13291520; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-17 20:24:42,136 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 8400896; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-17 20:24:52,143 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 8327168; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-17 20:25:02,150 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 8196096; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-17 20:25:12,159 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 8118272; capacity: 29394726912. Object creation will fail if spilli

(raylet) [2022-11-17 20:29:52,380 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 3293184; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-17 20:30:02,389 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 3174400; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-17 20:30:12,396 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 3092480; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-17 20:30:22,403 E 26936 26954] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-17_19-58-28_807464_26368 is over 95% full, available space: 3035136; capacity: 29394726912. Object creation will fail if spilli